In [61]:
import pymongo

# Connect to the MongoDB server
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Create or access a database
db = client["country"]


In [62]:
# Creating a database (it will only be created when we add a collection)
db = client["country"]
print("Database created successfully!")


Database created successfully!


In [63]:
# Creating a collection (table)
students = db["students"]
print("Collection created successfully!")


Collection created successfully!


In [64]:
# Insert a single document (record)
student_data = {
    "name": "John Doe",
    "age": 21,
    "department": "Computer Science"
}
students.insert_one(student_data)

# Insert multiple documents
students.insert_many([
    {"name": "Alice", "age": 22, "department": "AI"},
    {"name": "Bob", "age": 20, "department": "Data Science"}
])
print("Data inserted successfully!")


Data inserted successfully!


In [65]:
# Retrieve all documents
for student in students.find():
    print(student)



{'_id': ObjectId('679b9add4fd493a63bec0caf'), 'name': 'John Doe', 'age': 21, 'department': 'Computer Science'}
{'_id': ObjectId('679b9ade4fd493a63bec0cb0'), 'name': 'Alice', 'age': 22, 'department': 'AI'}
{'_id': ObjectId('679b9ade4fd493a63bec0cb1'), 'name': 'Bob', 'age': 20, 'department': 'Data Science'}


In [66]:
# Retrieve specific fields
for student in students.find({}, {"_id": 0, "name": 1, "department": 1}):
    print(student)

{'name': 'John Doe', 'department': 'Computer Science'}
{'name': 'Alice', 'department': 'AI'}
{'name': 'Bob', 'department': 'Data Science'}


In [67]:
# print single record
student = students.find_one({}, {"_id": 0, "name": 1, "department": 1})
print(student)


{'name': 'John Doe', 'department': 'Computer Science'}


In [68]:
# Skip the first 2 records and retrieve the third one
third_student = students.find().skip(2).limit(1)
for student in third_student:
    print(student)


{'_id': ObjectId('679b9ade4fd493a63bec0cb1'), 'name': 'Bob', 'age': 20, 'department': 'Data Science'}


In [69]:
# Find all students, sort by the "name" field, skip the first 2 records, and limit the result to the third one
third_student = students.find().sort("name").skip(2).limit(1)

# Loop through the cursor and print the third record (since limit(1) is used, it will only return one document)
for student in third_student:
    print(student)


{'_id': ObjectId('679b9add4fd493a63bec0caf'), 'name': 'John Doe', 'age': 21, 'department': 'Computer Science'}


In [70]:
# Find students older than 20
for student in students.find({"age": {"$gt": 20}}):
    print(student)


{'_id': ObjectId('679b9add4fd493a63bec0caf'), 'name': 'John Doe', 'age': 21, 'department': 'Computer Science'}
{'_id': ObjectId('679b9ade4fd493a63bec0cb0'), 'name': 'Alice', 'age': 22, 'department': 'AI'}


In [71]:
# Find students whose age is equal to 22
for student in students.find({"age": {"$eq": 22}}):
    print(student)


{'_id': ObjectId('679b9ade4fd493a63bec0cb0'), 'name': 'Alice', 'age': 22, 'department': 'AI'}


In [72]:
# Find students whose age is less than 20
for student in students.find({"age": {"$lt": 22}}):
    print(student)


{'_id': ObjectId('679b9add4fd493a63bec0caf'), 'name': 'John Doe', 'age': 21, 'department': 'Computer Science'}
{'_id': ObjectId('679b9ade4fd493a63bec0cb1'), 'name': 'Bob', 'age': 20, 'department': 'Data Science'}


In [73]:
# Sort by age in ascending order
for student in students.find().sort("age", 1):
    print(student)



{'_id': ObjectId('679b9ade4fd493a63bec0cb1'), 'name': 'Bob', 'age': 20, 'department': 'Data Science'}
{'_id': ObjectId('679b9add4fd493a63bec0caf'), 'name': 'John Doe', 'age': 21, 'department': 'Computer Science'}
{'_id': ObjectId('679b9ade4fd493a63bec0cb0'), 'name': 'Alice', 'age': 22, 'department': 'AI'}


In [74]:

# Sort by age in descending order
for student in students.find().sort("age", -1):
    print(student)

{'_id': ObjectId('679b9ade4fd493a63bec0cb0'), 'name': 'Alice', 'age': 22, 'department': 'AI'}
{'_id': ObjectId('679b9add4fd493a63bec0caf'), 'name': 'John Doe', 'age': 21, 'department': 'Computer Science'}
{'_id': ObjectId('679b9ade4fd493a63bec0cb1'), 'name': 'Bob', 'age': 20, 'department': 'Data Science'}


In [75]:
# # Drop a collection
# students.drop()
# print("Collection dropped successfully!")


In [77]:
# Update a single document
students.update_one({"name": "John Doe"}, {"$set": {"age": 23}})


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [78]:
# Update multiple documents
students.update_many({"department": "Computer Science"}, {"$set": {"department": "CS"}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [81]:
# LIMITS
# Retrieve only 2 documents
for student in students.find().limit(2):
    print(student)


{'_id': ObjectId('679b9add4fd493a63bec0caf'), 'name': 'John Doe', 'age': 23, 'department': 'CS'}
{'_id': ObjectId('679b9ade4fd493a63bec0cb0'), 'name': 'Alice', 'age': 22, 'department': 'AI'}


In [82]:
# JOIN 
# Creating another collection for demonstration
courses = db["courses"]
courses.insert_many([
    {"student_name": "John Doe", "course": "Database Systems"},
    {"student_name": "Alice", "course": "Machine Learning"}
])

# Aggregation with $lookup (Joining students with courses)
result = students.aggregate([
    {
        "$lookup": {
            "from": "courses",
            "localField": "name",
            "foreignField": "student_name",
            "as": "enrolled_courses"
        }
    }
])

for doc in result:
    print(doc)


{'_id': ObjectId('679b9add4fd493a63bec0caf'), 'name': 'John Doe', 'age': 23, 'department': 'CS', 'enrolled_courses': []}
{'_id': ObjectId('679b9ade4fd493a63bec0cb0'), 'name': 'Alice', 'age': 22, 'department': 'AI', 'enrolled_courses': []}
{'_id': ObjectId('679b9ade4fd493a63bec0cb1'), 'name': 'Bob', 'age': 20, 'department': 'Data Science', 'enrolled_courses': []}


In [76]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Access the database
db = client["university"]

# Create a collection and insert a sample document
level = db["class"]
level.insert_one({"name": "Level7", "department": "IT"})

# Now check the database and collection names
print(client.list_database_names())  # "university" should now appear
print(db.list_collection_names())  # "class" should now appear


['admin', 'config', 'country', 'kigali', 'local', 'university']
['class']
